### Configure and Authenticate

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION = "<SUBSCRIPTION_ID>"
RESOURCE_GROUP = "<RESOURCE_GROUP>"
WS_NAME = "<WORKSPACE>"
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [ ]:
# Verify that the handle works correctly.
ws = ml_client.workspaces.get(WS_NAME)
print(ws.location, ":", ws.resource_group)

### Create Custom Environment

Environment creation is executed only once or whenever update is required within environment. Uncomment if required.

In [ ]:
from azure.ai.ml.entities import Environment, BuildContext

env_docker_context = Environment(
    build=BuildContext(path="src/docker-contexts/stock-predictor"),
    name="stock_predictor_env",
    description="Custom Stock Predictor environment",
)
ml_client.environments.create_or_update(env_docker_context)

In [ ]:
# Check environment (can also be viewed in the custom environments section)
env = ml_client.environments.get(name="stock_predictor_env", label="latest")
print(env.version)

In [ ]:
# Get latest environment
environment_name = "stock_predictor_env"
env = ml_client.environments.get(name=environment_name, label="latest")
environment = environment_name + ":" + env.version

### Prepare and submit training job

In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Input
import datetime

epoch = datetime.datetime.now().timestamp()
job_name = "train_stock_predictor_" + str(int(epoch))

training_job = command(
    name=job_name,
    inputs=dict(
        previous_time_window=10,
        next_time_window=5,
        num_epochs=1,
        learning_rate=0.01,
        output_dir="./outputs",
        mode="train",
    ),
    code="./src/",
    command="python train.py --previous_time_window ${{inputs.previous_time_window}} --next_time_window ${{inputs.next_time_window}} --num_epochs ${{inputs.num_epochs}} --learning_rate ${{inputs.learning_rate}} --output_dir ${{inputs.output_dir}} --mode ${{inputs.mode}}",
    environment=environment,
    display_name="stock_predictor_training",
)

In [ ]:
ml_client.create_or_update(training_job)

In [ ]:
# Get jobs
jobs = ml_client.jobs.list()
print(list(jobs)[0])

### Register saved torch model

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

registered_model_name = "stock_predictor"

cloud_model = Model(
    path="azureml://jobs/{}/outputs/artifacts/paths/outputs/".format(job_name),
    name=registered_model_name,
    type=AssetTypes.CUSTOM_MODEL,
    description="Stock Predictor Model",
)
ml_client.models.create_or_update(cloud_model)